In [3]:
#Import modules
import numpy as np 
import pandas as pd 
import sqlite3
import matplotlib.pyplot as plt

In [9]:
# connecting sqlite database 
conn = sqlite3.connect('SalesAnalysis.sqlite3')

# to ignore Str decoding error
conn.text_factory = lambda b: b.decode(errors = 'ignore')


In [15]:
# display all tables in db

table = pd.read_sql("""
                        SELECT * FROM sqlite_master 
                        WHERE  type ='table' AND 
                            name NOT LIKE 'sqlite_%';
                        """, conn)
table


,type,name,tbl_name,rootpage,sql
0,table,Categories,Categories,2,CREATE TABLE `Categories` (\r\n\r\nCategoryID ...
1,table,CustomerCustomerDemo,CustomerCustomerDemo,4,CREATE TABLE `CustomerCustomerDemo` (\r\n\r\nC...
2,table,CustomerDemographics,CustomerDemographics,6,CREATE TABLE `CustomerDemographics` (\r\n\r\nC...
3,table,Customers,Customers,8,CREATE TABLE `Customers` (\r\n\r\nCustomerID v...
4,table,Employees,Employees,12,CREATE TABLE `Employees` (\r\n\r\nEmployeeID i...
5,table,EmployeeTerritories,EmployeeTerritories,14,CREATE TABLE `EmployeeTerritories` (\r\n\r\nEm...
6,table,OrderDetails,OrderDetails,17,"CREATE TABLE ""OrderDetails"" (\r\n\r\nOrderID i..."
7,table,Products,Products,21,CREATE TABLE `Products` (\r\n\r\nProductID int...
8,table,Region,Region,24,CREATE TABLE `Region` (\r\n\r\nRegionID int NO...
9,table,Shippers,Shippers,26,CREATE TABLE `Shippers` (\r\n\r\nShipperID int...


In [7]:
# To know schema of table 'Orders'
table_schema = pd.read_sql("""
                        PRAGMA table_info(Orders);
                        """, conn)
table_schema

,cid,name,type,notnull,dflt_value,pk
0,0,OrderId,INT,0,None,0
1,1,CustomerID,TEXT,0,None,0
2,2,EmployeeID,INT,0,None,0
3,3,OrderDate,,0,None,0
4,4,ShippedDate,,0,None,0
5,5,RequiredDate,,0,None,0
6,6,ShipVia,INT,0,None,0
7,7,Freight,REAL,0,None,0
8,8,ShipName,TEXT,0,None,0
9,9,ShipAddress,TEXT,0,None,0


In [73]:
# read data using sql query
Order_table = pd.read_sql("""SELECT *
                        FROM Orders;""", conn)
Order_table

,OrderId,CustomerID,EmployeeID,OrderDate,ShippedDate,RequiredDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2014-07-04,2014-07-16,2014-08-01,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
1,10249,TOMSP,6,2014-07-05,2014-07-10,2014-08-16,1,11.61,Toms Spezialitten,Luisenstr. 48,Mnster,None,44087,Germany
2,10250,HANAR,4,2014-07-08,2014-07-12,2014-08-05,2,65.83,Hanari Carnes,"Rua do Pao, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,2014-07-08,2014-07-15,2014-08-05,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
4,10252,SUPRD,4,2014-07-09,2014-07-11,2014-08-06,2,51.30,Suprmes dlices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,11073,PERIC,2,2016-05-05,None,2016-06-02,2,24.95,Pericles Comidas clsicas,Calle Dr. Jorge Cash 321,Mxico D.F.,None,05033,Mexico
826,11074,SIMOB,7,2016-05-06,None,2016-06-03,2,18.44,Simons bistro,Vinbltet 34,Kobenhavn,None,1734,Denmark
827,11075,RICSU,8,2016-05-06,None,2016-06-03,2,6.19,Richter Supermarkt,Starenweg 5,Genve,None,1204,Switzerland
828,11076,BONAP,4,2016-05-06,None,2016-06-03,2,38.28,Bon app',"12, rue des Bouchers",Marseille,None,13008,France


### 1. Top 5 cities with total number of customer orders

In [8]:
top_cities_orders = pd.read_sql("""
                                    SELECT o.OrderId,c.CustomerID,c.City,count(*) Total_Orders
                                    FROM Orders o
                                    JOIN Customers c
                                    ON c.CustomerID=o.CustomerID
                                    GROUP BY City
                                    ORDER BY count(*) DESC
                                    LIMIT 5
                                    """, conn)
top_cities_orders

,OrderId,CustomerID,City,Total_Orders
0,10289,BSBEV,London,46
1,10250,HANAR,Rio de Janeiro,34
2,10324,SAVEA,Boise,31
3,10290,COMMI,Sao Paulo,31
4,10258,ERNSH,Graz,30


### 2. Top 5 cities for total customer purchases

In [101]:
top_cities_quantity = pd.read_sql("""
                                   SELECT o.OrderId,c.CustomerID,c.City,sum(Quantity)Total_quantity                                 
                                    FROM Orders o
                                    JOIN Customers c
                                    ON c.CustomerID=o.CustomerID
                                    JOIN OrderDetails od
                                    on o.OrderId=od.OrderId
                                    GROUP BY City
                                    ORDER BY Total_quantity DESC
                                    LIMIT 5
                                    """, conn)
top_cities_quantity

,OrderId,CustomerID,City,Total_quantity
0,10324,SAVEA,Boise,4958
1,10258,ERNSH,Graz,4543
2,10273,QUICK,Cunewalde,3961
3,10289,BSBEV,London,2447
4,10250,HANAR,Rio de Janeiro,1893


### 3. Top 3 customers with the largest purchase volume

In [102]:
customer_purchases_vol = pd.read_sql("""
                                   SELECT c.CompanyName,sum(Quantity)Total_quantity,c.City,c.Country
                                    FROM Orders o
                                    JOIN Customers c
                                    ON c.CustomerID=o.CustomerID
                                    JOIN OrderDetails od
                                    on o.OrderId=od.OrderId
                                    GROUP BY CompanyName
                                    ORDER BY Total_quantity DESC
                                    LIMIT 5
                                    """, conn)
customer_purchases_vol

,CompanyName,Total_quantity,City,Country
0,Save-a-lot Markets,4958,Boise,USA
1,Ernst Handel,4543,Graz,Austria
2,QUICK-Stop,3961,Cunewalde,Germany
3,Hungry Owl All-Night Grocers,1684,Cork,Ireland
4,Frankenversand,1525,Mnchen,Germany


### 4.  Customer monthly average order quantity ranking

In [106]:
avg_quantity_orders = pd.read_sql("""
                                   SELECT CompanyName,Avg(Total_quantity)AS Avg_Monthly_quantity
                                    FROM(SELECT STRFTIME('%Y-%m', o.OrderDate) ord_date,c.CompanyName,sum(Quantity)Total_quantity
                                    FROM orders o 
                                    JOIN Customers c
                                    ON o.CustomerID=c.CustomerID
                                    JOIN OrderDetails od
                                    ON O.OrderId=od.OrderId
                                    GROUP BY CompanyName,ord_date) sub
                                    GROUP BY 1
                                    ORDER BY 2 DESC
                                    """, conn)
avg_quantity_orders

,CompanyName,Avg_Monthly_quantity
0,Save-a-lot Markets,330.533333
1,Ernst Handel,267.235294
2,QUICK-Stop,247.562500
3,Bottom-Dollar Markets,159.333333
4,Seven Seas Imports,136.333333
...,...,...
84,Centro comercial Moctezuma,11.000000
85,Franchi S.p.A.,10.800000
86,Lazy K Kountry Store,10.000000
87,North/South,10.000000


### 5. What are the five most popular products?

In [105]:
popular_products = pd.read_sql("""
                                   SELECT p.ProductName,SUM(od.Quantity) Total_Sales, c.CategoryName, s.Country
                                    FROM Products p 
                                    JOIN OrderDetails od 
                                    ON od.ProductID=p.ProductID 
                                    JOIN Orders o 
                                    ON o.OrderId=od.OrderId 
                                    JOIN Categories c 
                                    ON c.CategoryID=p.CategoryID 
                                    JOIN Suppliers s
                                    On p.SupplierID = s.SupplierID
                                    GROUP BY 1 
                                    ORDER BY 2 DESC 
                                    LIMIT 5;
                                    """, conn)
popular_products

,ProductName,Total_Sales,CategoryName,Country
0,Camembert Pierrot,1577,Dairy Products,France
1,Raclette Courdavault,1496,Dairy Products,France
2,Gorgonzola Telino,1397,Dairy Products,Italy
3,Gnocchi di nonna Alice,1263,Grains/Cereals,Italy
4,Pavlova,1158,Confections,Australia


### 6. Which companies are selling more products?

In [58]:
top_customer = pd.read_sql("""
                                   SELECT  t1.SupplierID, t1.CompanyName, t1.Country, round(t1.total_sales * 100.00 /t2.total, 3) || '%'  percent
                                    FROM 
                                    (SELECT p.SupplierID, s.CompanyName, SUM(od.Quantity) total_sales, s.Country
                                    FROM Products p
                                    JOIN Suppliers s
                                    ON p.SupplierID = s.SupplierID
                                    JOIN OrderDetails od 
                                    ON od.ProductID=p.ProductID 
                                    JOIN Orders o 
                                    ON o.OrderId=od.OrderId 
                                    GROUP BY 1
                                    ORDER BY 3 DESC ) t1,
                                    (SELECT SUM(total_sales) total
                                    FROM 
                                    (SELECT p.SupplierID, s.CompanyName, SUM(od.Quantity) total_sales
                                    FROM Products p
                                    JOIN Suppliers s
                                    ON p.SupplierID = s.SupplierID
                                    JOIN OrderDetails od 
                                    ON od.ProductID=p.ProductID 
                                    JOIN Orders o 
                                    ON o.OrderId=od.OrderId 
                                    GROUP BY 1
                                    ORDER BY 3 DESC )) t2
                                    LIMIT 10;
                                    """, conn)
top_customer

,SupplierID,CompanyName,Country,percent
0,12,Plutzer Lebensmittelgromrkte AG,Germany,7.935%
1,7,"Pavlova, Ltd.",Australia,7.672%
2,28,Gai pturage,France,5.988%
3,8,"Specialty Biscuits, Ltd.",UK,5.556%
4,15,Norske Meierier,Norway,4.922%
5,14,Formaggi Fortini s.r.l.,Italy,4.872%
6,1,Exotic Liquids,UK,4.312%
7,24,"G'day, Mate",Australia,4.108%
8,19,New England Seafood Cannery,USA,4.061%
9,20,Leka Trading,Singapore,3.66%


### 7. What is the category with the largest number and the smallest number of products?

In [60]:
top_category_products = pd.read_sql("""
                                   SELECT CategoryName, count(*) num_products FROM Products p
                                    JOIN Categories c
                                    ON p.CategoryID = c.CategoryID
                                    GROUP BY CategoryName
                                    ORDER BY num_products DESC
                                    """, conn)
top_category_products

,CategoryName,num_products
0,Confections,13
1,Beverages,12
2,Condiments,12
3,Seafood,12
4,Dairy Products,10
5,Grains/Cereals,7
6,Meat/Poultry,6
7,Produce,5


### 8. Which categories have more sales than the average value?

In [59]:
top_category = pd.read_sql("""
                                   SELECT CategoryName, COUNT(*) total_sales FROM Products p
                                    JOIN Categories c
                                    ON p.CategoryID = c.CategoryID
                                    JOIN OrderDetails o
                                    On o.ProductID = p.ProductID
                                    GROUP BY CategoryName

                                    """, conn)
top_category

,CategoryName,total_sales
0,Beverages,404
1,Condiments,216
2,Confections,334
3,Dairy Products,366
4,Grains/Cereals,196
5,Meat/Poultry,173
6,Produce,136
7,Seafood,330
